In [12]:
!pip install pathway bokeh panel --quiet

In [13]:
import pandas as pd
import pathway as pw
import panel as pn
import bokeh.plotting
import datetime




In [14]:
# Load CSV and combine timestamp
df = pd.read_csv("/content/dataset.csv")
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                 format="%d-%m-%Y %H:%M:%S")
df = df.sort_values("Timestamp").reset_index(drop=True)



In [15]:
# Save minimal fields for streaming
df[["Timestamp", "Occupancy", "Capacity", "QueueLength", "TrafficConditionNearby",
    "VehicleType", "IsSpecialDay"]].to_csv("parking_stream_model2.csv", index=False)

# Schema
class Model2Schema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    VehicleType: str
    IsSpecialDay: int



In [16]:
# Replay as stream
stream = pw.demo.replay_csv("parking_stream_model2.csv", schema=Model2Schema, input_rate=1000)

# Set weights
ALPHA, BETA, GAMMA, DELTA, EPSILON = 2, 0.5, 1, 2, 1.5
BASE_PRICE, LAMBDA = 10, 1.0
fmt = "%Y-%m-%d %H:%M:%S"



In [17]:
# Preprocess and calculate demand
data = stream.with_columns(
    t = stream.Timestamp.dt.strptime(fmt),
    occ_ratio = stream.Occupancy / stream.Capacity,
    traffic = pw.if_else(
        stream.TrafficConditionNearby == "low", 0,
        pw.if_else(
            stream.TrafficConditionNearby == "average", 1,
            pw.if_else(stream.TrafficConditionNearby == "high", 2, 1)
        )
    ),
    vehicle_weight = pw.if_else(
        stream.VehicleType == "cycle", 0.5,
        pw.if_else(
            stream.VehicleType == "bike", 1.0,
            pw.if_else(
                stream.VehicleType == "car", 2.0,
                pw.if_else(stream.VehicleType == "truck", 3.0, 1.0)
            )
        )
    )
).with_columns(
    raw_demand = (
        ALPHA * pw.this.occ_ratio +
        BETA * pw.this.QueueLength -
        GAMMA * pw.this.traffic +
        DELTA * stream.IsSpecialDay +
        EPSILON * pw.this.vehicle_weight
    )
)



In [18]:

# Min-max normalization window
norm = data.windowby(
    pw.this.t,
    window=pw.temporal.sliding(
        duration=datetime.timedelta(minutes=30),
        hop=datetime.timedelta(minutes=30)
    ),
    behavior=pw.temporal.exactly_once_behavior()
).reduce(
    t = pw.this._pw_window_end,
    min_d = pw.reducers.min(pw.this.raw_demand),
    max_d = pw.reducers.max(pw.this.raw_demand),
)



In [19]:

# Join to get normalized price
# --- Join and Flatten ---
joined = data.join(norm, pw.left.t == pw.right.t).select(
    t = pw.left.t,
    raw_demand = pw.left.raw_demand,
    min_d = pw.right.min_d,
    max_d = pw.right.max_d
)

# --- Step 1: Normalize Demand ---
joined = joined.with_columns(
    norm_demand = (pw.this.raw_demand - pw.this.min_d) / (pw.this.max_d - pw.this.min_d + 1e-5)
)

# --- Step 2: Compute Price and Clamp ---
# --- Step 2.1: Compute Raw Price ---
step2a = joined.with_columns(
    price = BASE_PRICE * (1 + LAMBDA * pw.this.norm_demand)
)

# --- Step 2.2: Clamp Price Safely ---
step2b = step2a.with_columns(
    price_clamped = pw.apply(
        lambda p: max(0.5 * BASE_PRICE, min(p, 2 * BASE_PRICE)),
        pw.this.price
    )
)



In [20]:
# --- Final Output Table ---
final = step2b.select(
    t = pw.this.t,
    price_clamped = pw.this.price_clamped
)



In [21]:
# Plot
pn.extension()
def plot(source):
    fig = bokeh.plotting.figure(x_axis_type="datetime", height=400, width=800,
                                title="Model 2: Demand-Based Dynamic Pricing")
    fig.line("t", "price_clamped", source=source, line_width=2, color="green")
    fig.circle("t", "price_clamped", source=source, size=5, color="orange")
    return fig

viz = final.plot(plot, sorting_col="t")
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [22]:

# Start engine
pw.run()

Output()